In [ ]:
"""
Created on Thu Oct 24 09:54 2024

Try to prepare grid for remapcon using xesmf

@author: Clara Burgard
"""

In [1]:
import xarray as xr
import xesmf
import numpy as np

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
path1 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/'

In [5]:
# Load your curvilinear grid file
ds = xr.open_dataset(path1 + 'coordinates.nc')

In [18]:
# Assuming the dataset has 'lat' and 'lon' variables
lat = ds['gphit']  # 2D array of latitudes
lon = ds['glamt'] # 2D array of longitudes
# Create a longitude with 0-360 values to avoir a gap in the bounds later
lon360 = lon.where(lon>0,lon+360)

In [27]:
# Create the lat and lon of the grid corners : ll = lower left, lr = lower right, ur = upper right, ul = upper left
meshlon_ll,meshlat_ll = (lon + lon.roll(x=-1))/2, (lat + lat.roll(x=-1))/2
meshlon_lr,meshlat_lr = (lon + lon.roll(x=1))/2, (lat + lat.roll(x=-1))/2
meshlon_ur,meshlat_ur = (lon + lon.roll(x=1))/2, (lat + lat.roll(x=1))/2
meshlon_ul,meshlat_ul = (lon + lon.roll(x=-1))/2, (lat + lat.roll(x=1))/2

In [28]:
# Create the lon of the grid corners using 0-360 : ll = lower left, lr = lower right, ur = upper right, ul = upper left
meshlon360_ll,_ = (lon360 + lon360.roll(x=-1))/2, (lat + lat.roll(x=-1))/2
meshlon360_lr,_ = (lon360 + lon360.roll(x=1))/2, (lat + lat.roll(x=-1))/2
meshlon360_ur,_ = (lon360 + lon360.roll(x=1))/2, (lat + lat.roll(x=1))/2
meshlon360_ul,_ = (lon360 + lon360.roll(x=-1))/2, (lat + lat.roll(x=1))/2

In [29]:
lon_bnds = xr.DataArray(data=np.array([meshlon_ll, meshlon_lr, meshlon_ur, meshlon_ul]), dims=['nvertex','y','x'])
lat_bnds = xr.DataArray(data=np.array([meshlat_ll, meshlat_lr, meshlat_ur, meshlat_ul]), dims=['nvertex','y','x'])
lon360_bnds = xr.DataArray(data=np.array([meshlon360_ll, meshlon360_lr, meshlon360_ur, meshlon360_ul]), dims=['nvertex','y','x']) 

In [30]:
# region where longitude hops from 180 to -180
prob_region = (lon <= -140) | (lon >= 140) & (np.abs(lon_bnds) < 25)
new_lon_bnds = lon_bnds.where(~prob_region, lon360_bnds)
new_lon_bnds = new_lon_bnds.where(new_lon_bnds <= 180, new_lon_bnds - 360)

In [31]:
NEMO_new_grid = xr.merge([new_lon_bnds.rename('lon_bnds'),
                          lat_bnds.rename('lat_bnds')]).assign_coords({'lon': lon, 'lat': lat})

In [32]:
NEMO_new_grid['lon'].attrs['standard_name'] = 'longitude'
NEMO_new_grid['lon'].attrs['units'] = 'degrees_east'
NEMO_new_grid['lon'].attrs['bounds'] = 'lon_bnds'

NEMO_new_grid['lat'].attrs['standard_name'] = 'latitude'
NEMO_new_grid['lat'].attrs['units'] = 'degrees_north'
NEMO_new_grid['lat'].attrs['bounds'] = 'lat_bnds'


In [33]:
NEMO_new_grid[['lon','lat','lon_bnds','lat_bnds']].set_coords(
    ("lat_bnds", "lon_bnds")).transpose('y','x','nvertex').to_netcdf(inputpath + 'NEMO_grid_withbnds.nc')